In [1]:
# Create the contrast based on i1 and i2

In [2]:
!pwd
!date

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/MovieMitosis
Sun Jun 26 02:43:47 PDT 2022


In [3]:
import os
import sys
import json
import importlib
import concurrent
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from skimage import io as skio
import matplotlib.pyplot as plt
from aicscytoparam import cytoparam
from aicsshparam import shtools, shparam
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from cvapipe_analysis.tools import io, viz, general, controller, shapespace, plotting

sys.path.insert(1, '../tools')
import common

In [4]:
path_config = Path("/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/")
control_main = controller.Controller(general.load_config_file(path_config))
device_main = io.LocalStagingIO(control_main)
print(control_main.get_staging())

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_variance


In [5]:
df = device_main.load_step_manifest("loaddata")
print(df.shape)

(216062, 46)


### Create domain (mean cell shape)

In [6]:
datasets = ["m1", "m2"]
genes = control_main.get_gene_names()

In [7]:
morphs = dict([(gene, []) for gene in genes])
for ds in datasets:
    inner_alias = control_main.get_inner_most_alias_to_parameterize()
    outer_alias = control_main.get_outer_most_alias_to_parameterize()
    inner_mesh = device_main.read_vtk_polydata(f"avgshape/{ds}_{inner_alias}_matched.vtk")
    outer_mesh = device_main.read_vtk_polydata(f"avgshape/{ds}_{outer_alias}_matched.vtk")
    
    staging = "m1m2" if ds=="m1" else "m3"
    path_step = Path(f"/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_variance_{staging}/shapemode")
    control, device = common.get_managers_from_step_path(path_step)

    row = pd.Series({"shape_mode": "NUC_MEM_PC1", "mpId": 1, "aggtype": "avg", "alias": "STR"})
    domain, domain_nuc, domain_mem, coords_param = common.get_map_point_shape(control, device, row, inner_mesh=inner_mesh, outer_mesh=outer_mesh)
    
    for gene in genes:
        row["structure"] = gene
        rep = device.read_agg_parameterized_intensity(row)
        morphed = cytoparam.morph_representation_on_shape(
            img=domain,
            param_img_coords=coords_param,
            representation=rep
        )
        morphed = np.stack([domain_nuc, domain_mem, morphed], axis=0)
        morphs[gene].append(morphed)

In [8]:
importlib.reload(common)

<module 'common' from '/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/MovieMitosis/../tools/common.py'>

In [12]:
contrast = {}
for gene in genes:
    mode = {"nuc":"center_nuc","mem":"center_nuc","gfp":"center_nuc"}
    contrast[gene] = common.Projector.get_shared_morphed_max_based_on_pct_for_zy_views(
        instances = morphs[gene],
        pct = 90,
        mode = mode,
        func = np.mean
    )

In [13]:
log = {"contrast": contrast, "version": {}}
for fname in ["../tools/common.py", "MakeContrastTable.ipynb"]:
    with open(fname, "r") as ftxt:
        log["version"][fname] = ftxt.read()
with open("contrast_V3_90OnValids.json", "w") as fj:
    json.dump(log, fj, indent=4)

In [14]:
common.now("complete")

complete 2022-06-26 03:05:34
